<a href="https://colab.research.google.com/github/srnarasim/TAOExperiment/blob/main/TAOExperiment_LLM_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAO Experiment - Text Classification with LLM and LoRA

This notebook implements Test-time Adaptation for Out-of-distribution detection (TAO) using:
1. Fine-tuning a pre-trained LLM (Llama-2-7b) on known product categories
2. Implementing test-time adaptation to dynamically handle new categories
3. Using LoRA for lightweight updates when new categories emerge

## 1. Setup and Dependencies

In [1]:
# Install required packages
!pip install -q trl transformers datasets accelerate bitsandbytes peft torch pandas numpy scikit-learn --upgrade


In [2]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
from trl import SFTTrainer

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 2. Create Sample Dataset

In [3]:
# Create a CSV file with sample content
import csv

data = [
    ['Product', 'Product Description', 'Category'],
    ['Wireless Bluetooth headphones with noise cancellation', 'Headphones', 'Electronics'],
    ['Smartphone with OLED display and 128GB storage', 'Smartphone', 'Electronics'],
    ['Gaming laptop with high refresh rate screen', 'Laptop', 'Electronics'],
    ['Smart home security camera with night vision', 'Smart Home Device', 'Electronics'],
    ['Cotton t-shirt with graphic print design', 'T-shirt', 'Clothing'],
    ['Denim jeans with distressed finish', 'Jeans', 'Clothing'],
    ['Wool sweater with cable knit pattern', 'Sweater', 'Clothing'],
    ['Leather jacket with quilted lining', 'Jacket', 'Clothing'],
    ['Wooden dining table with six matching chairs', 'Dining Table', 'Furniture'],
    ['Memory foam mattress with cooling gel', 'Mattress', 'Furniture'],
    ['Bookshelf with adjustable shelves', 'Bookshelf', 'Furniture'],
    ['Leather sofa with reclining function', 'Sofa', 'Furniture'],
    ['Genuine leather wallet with multiple card slots', 'Wallet', 'Accessories'],
    ['Stainless steel watch with leather strap', 'Watch', 'Accessories'],
    ['Polarized sunglasses with UV protection', 'Sunglasses', 'Accessories'],
    ['Silk scarf with floral pattern', 'Scarf', 'Accessories'],
    ['Insulated stainless steel water bottle', 'Water Bottle', 'Kitchen'],
    ['Non-stick frying pan with glass lid', 'Frying Pan', 'Kitchen'],
    ['Electric coffee maker with programmable timer', 'Coffee Maker', 'Kitchen'],
    ['Ceramic dinner plate set with modern design', 'Plate Set', 'Kitchen']
]

with open('balanced_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

# Display the dataset
df = pd.read_csv('balanced_data.csv')
print(f"Dataset shape: {df.shape}")
print(f"Categories: {df['Category'].unique()}")
df.head()

Dataset shape: (20, 3)
Categories: ['Electronics' 'Clothing' 'Furniture' 'Accessories' 'Kitchen']


,Product,Product Description,Category
0,Wireless Bluetooth headphones with noise cance...,Headphones,Electronics
1,Smartphone with OLED display and 128GB storage,Smartphone,Electronics
2,Gaming laptop with high refresh rate screen,Laptop,Electronics
3,Smart home security camera with night vision,Smart Home Device,Electronics
4,Cotton t-shirt with graphic print design,T-shirt,Clothing


## 3. Prepare Data for LLM Fine-tuning

In [4]:
# Format data for instruction fine-tuning
def format_instruction(product):
    return f"""### Instruction:
Classify the following product into one of these categories: Electronics, Clothing, Furniture, Accessories, Kitchen.

### Product:
{product}

### Response:
"""

# Prepare dataset
df = df.iloc[1:].reset_index(drop=True)  # Remove header row if it was included
df['instruction'] = df['Product'].apply(format_instruction)
df['response'] = df['Category']

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['Category'], random_state=42)

# Convert to HuggingFace datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

# Display a sample
print("\nSample instruction:")
print(train_dataset[0]['instruction'])
print(f"Expected response: {train_dataset[0]['response']}")

Training samples: 13
Testing samples: 6

Sample instruction:
### Instruction: 
Classify the following product into one of these categories: Electronics, Clothing, Furniture, Accessories, Kitchen.

### Product:
Memory foam mattress with cooling gel

### Response:

Expected response: Furniture


## 4. Load Pre-trained LLM and Configure LoRA

In [5]:
# Define model name
model_name = "meta-llama/Llama-2-7b-hf"  # You can also use "TinyLlama/TinyLlama-1.1B-Chat-v1.0" for faster training

# Configure quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right")
tokenizer.pad_token = tokenizer.eos_token

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)
print("Model loaded with LoRA configuration")
print(model.print_trainable_parameters())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded with LoRA configuration
trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.2484
None


## 5. Fine-tune the Model on Known Categories

In [6]:
#!pip install -q trl transformers datasets accelerate bitsandbytes peft torch pandas numpy scikit-learn --upgrade

In [7]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results/llm-product-classifier",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    report_to="none"
)

# Preprocess your dataset to have the expected input format
def preprocess_train_dataset(example):
    return {"text": example["instruction"]}

# Apply the preprocessing to your dataset
processed_train_dataset = train_dataset.map(preprocess_train_dataset)

# Initialize SFT trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=processed_train_dataset,
    args=training_args,
    peft_config=lora_config
)

def format_instruction(example):
    """Format instruction for training."""
    return example["instruction"]


# Custom tokenize function to handle "instruction" key
def tokenize(example, processing_class):
    processed = processing_class(text=example["instruction"])  # Pass "instruction"
    if (
        processing_class.eos_token_id is not None
        and processing_class.padding_side == "right"
    ):
        processed["labels"] = [
            -100 if i == processing_class.eos_token_id else i
            for i in processed["input_ids"]
        ]
    return processed

trainer.train()

# Save the fine-tuned model
model.save_pretrained("./results/llm-product-classifier-final")
tokenizer.save_pretrained("./results/llm-product-classifier-final")
print("Model training completed and saved")

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/13 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/13 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/13 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/13 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


Model training completed and saved


## 6. Evaluate the Model on Known Categories

In [8]:
# Load the fine-tuned model
peft_model_path = "./results/llm-product-classifier-final"
config = PeftConfig.from_pretrained(peft_model_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_path)

# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=20,
    temperature=0.1
)

def format_instruction(product):
    """Format product text into instruction for inference."""
    return f"### Instruction:\nCategorize the following product into one of the standard e-commerce categories.\n\n### Product:\n{product}\n\n### Response:\n"

# Function to predict category
def predict_category(product):
    prompt = format_instruction(product)
    response = pipe(prompt)[0]['generated_text']
    # Extract the category from the response
    category = response.split("### Response:\n")[-1].strip()
    return category

# Evaluate on test set
true_categories = []
predicted_categories = []

for i, sample in enumerate(test_dataset):
    product = sample['Product']
    true_category = sample['Category']
    predicted_category = predict_category(product)

    true_categories.append(true_category)
    predicted_categories.append(predicted_category)

    print(f"Product: {product}")
    print(f"True: {true_category}, Predicted: {predicted_category}")
    print("-" * 50)

# Calculate accuracy
accuracy = accuracy_score(true_categories, predicted_categories)
print(f"\nAccuracy on known categories: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(true_categories, predicted_categories))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

Product: Wool sweater with cable knit pattern
True: Clothing, Predicted: Clothing

### Instruction:
Categorize the following product into one of the
--------------------------------------------------
Product: Smart home security camera with night vision
True: Electronics, Predicted: ### Instruction:

Categorize the following product into one of the standard e
--------------------------------------------------
Product: Ceramic dinner plate set with modern design
True: Kitchen, Predicted: Home & Garden

### Instruction:
Categorize the following product into one of
--------------------------------------------------
Product: Polarized sunglasses with UV protection
True: Accessories, Predicted: ### Instruction:

Categorize the following product into one of the standard e
--------------------------------------------------
Product: Wooden dining table with six matching chairs
True: Furniture, Predicted: Furniture

### Instruction:
Categorize the following product into one of
-----------------

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

## 7. Implement Test-time Adaptation (TAO) for New Categories

In [9]:
import torch.nn.functional as F

# Function to calculate entropy from model outputs
def calculate_entropy(logits):
    # Get probabilities using softmax
    probs = F.softmax(logits, dim=-1)
    # Calculate entropy
    entropy = -torch.sum(probs * torch.log(probs + 1e-9), dim=-1)
    return entropy.mean().item()

# Function to predict with uncertainty estimation
def predict_with_tao(product, entropy_threshold=5.0):
    prompt = format_instruction(product)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        # Calculate entropy on the last token's logits
        last_token_logits = logits[:, -1, :]
        entropy = calculate_entropy(last_token_logits)

        # Generate response
        response = pipe(prompt)[0]['generated_text']
        category = response.split("### Response:\n")[-1].strip()

        # Check if entropy is high (uncertain prediction)
        if entropy > entropy_threshold:
            return "New Category", entropy
        else:
            return category, entropy

# Test with known and new products
test_products = [
    # Known categories
    "Wireless gaming headphones with RGB lighting",
    "Wooden dining table with extendable leaf",
    "Classic leather wallet with coin pocket",
    "Cotton polo shirt with embroidered logo",

    # Potentially new categories
    "Smart fitness tracker with heart rate monitor",  # Could be new (Wearable Tech)
    "Electric scooter with foldable design",          # Could be new (Transportation)
    "Organic vitamin supplements in glass bottle",    # New (Health & Wellness)
    "Handcrafted ceramic plant pot with drainage"     # New (Home & Garden)
]

print("Testing product classification with test-time adaptation:\n")
for product in test_products:
    category, entropy = predict_with_tao(product)
    print(f"Product: '{product}'")
    print(f"Classification: {category}")
    print(f"Uncertainty (Entropy): {entropy:.4f}")
    print("-" * 70)

Testing product classification with test-time adaptation:

Product: 'Wireless gaming headphones with RGB lighting'
Classification: ### Instruction:

### Product:
A pair of sneakers
Uncertainty (Entropy): nan
----------------------------------------------------------------------
Product: 'Wooden dining table with extendable leaf'
Classification: Furniture

### Instruction:
Categorize the following product into one of
Uncertainty (Entropy): nan
----------------------------------------------------------------------
Product: 'Classic leather wallet with coin pocket'
Classification: ### Instruction:

Categorize the following product into one of the standard e
Uncertainty (Entropy): nan
----------------------------------------------------------------------


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Product: 'Cotton polo shirt with embroidered logo'
Classification: ### Instruction:

Categorize the following product into one of the standard e
Uncertainty (Entropy): nan
----------------------------------------------------------------------
Product: 'Smart fitness tracker with heart rate monitor'
Classification: ### Instruction:

Categorize the following product into one of the standard e
Uncertainty (Entropy): nan
----------------------------------------------------------------------
Product: 'Electric scooter with foldable design'
Classification: - [ ] Electronics
- [ ] Fashion
- [ ] Home
- [
Uncertainty (Entropy): nan
----------------------------------------------------------------------
Product: 'Organic vitamin supplements in glass bottle'
Classification: ### Instruction:

### Product:
Organic vitamin supplements
Uncertainty (Entropy): nan
----------------------------------------------------------------------
Product: 'Handcrafted ceramic plant pot with drainage'
Classification:

## 8. LoRA Adaptation for New Categories

In [10]:
# Create a dataset with new categories
new_categories_data = [
    ['Product', 'Product Description', 'Category'],
    ['Smart fitness tracker with heart rate monitor', 'Fitness Tracker', 'Wearable Tech'],
    ['Smartwatch with GPS and sleep tracking', 'Smartwatch', 'Wearable Tech'],
    ['Wireless earbuds with fitness tracking', 'Earbuds', 'Wearable Tech'],
    ['Health monitoring ring with sleep analysis', 'Smart Ring', 'Wearable Tech'],
    ['Electric scooter with foldable design', 'Electric Scooter', 'Transportation'],
    ['Electric bicycle with pedal assist', 'E-Bike', 'Transportation'],
    ['Hoverboard with LED lights', 'Hoverboard', 'Transportation'],
    ['Electric skateboard with remote control', 'E-Skateboard', 'Transportation'],
    ['Organic vitamin supplements in glass bottle', 'Vitamins', 'Health & Wellness'],
    ['Essential oil diffuser with LED lights', 'Diffuser', 'Health & Wellness'],
    ['Yoga mat with alignment markings', 'Yoga Mat', 'Health & Wellness'],
    ['Meditation cushion with organic filling', 'Meditation Cushion', 'Health & Wellness'],
    ['Handcrafted ceramic plant pot with drainage', 'Plant Pot', 'Home & Garden'],
    ['Indoor herb garden kit with grow light', 'Herb Garden', 'Home & Garden'],
    ['Gardening tool set with ergonomic handles', 'Garden Tools', 'Home & Garden'],
    ['Self-watering planter for indoor plants', 'Self-watering Planter', 'Home & Garden']
]

with open('new_categories_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(new_categories_data)

# Load and prepare the new categories dataset
new_df = pd.read_csv('new_categories_data.csv')
new_df = new_df.iloc[1:].reset_index(drop=True)  # Remove header row
new_df['instruction'] = new_df['Product'].apply(format_instruction)
new_df['response'] = new_df['Category']

# Convert to HuggingFace dataset
new_categories_dataset = Dataset.from_pandas(new_df)
print(f"New categories dataset size: {len(new_categories_dataset)}")
print(f"New categories: {new_df['Category'].unique()}")

New categories dataset size: 15
New categories: ['Wearable Tech' 'Transportation' 'Health & Wellness' 'Home & Garden']


In [11]:
# Alternative approach with explicit device mapping
device_map = {
    "transformer.word_embeddings": 0,
    "transformer.h": "auto",        # Auto distribute transformer layers
    "transformer.ln_f": 0,
    "lm_head": "0"                # Put lm_head on CPU to save GPU memory
}
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # Ensure bfloat16 computation
    bnb_4bit_use_double_quant=True,        # Optional: Further reduces memory
    llm_int8_skip_modules=["lm_head"]      # Exclude `lm_head` from quantization
)



base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto"  # Let `accelerate` handle memory allocation
)

# Apply the new LoRA config
new_lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    modules_to_save=["lm_head"]
)

model_for_new_categories = get_peft_model(base_model, new_lora_config)
print("Model prepared with new LoRA adapter for new categories")
print(model_for_new_categories.print_trainable_parameters())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model prepared with new LoRA adapter for new categories
trainable params: 147,849,216 || all params: 6,886,264,832 || trainable%: 2.1470
None


In [12]:
# Define model name
model_name = "meta-llama/Llama-2-7b-hf"

# Configure quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Configure LoRA
new_lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model_for_new_categories = get_peft_model(base_model, new_lora_config)
print("Model prepared with new LoRA adapter for new categories")
print(model_for_new_categories.print_trainable_parameters())


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Model prepared with new LoRA adapter for new categories
trainable params: 16,777,216 || all params: 6,886,264,832 || trainable%: 0.2436
None


In [13]:
# Define training arguments for the new categories
new_training_args = TrainingArguments(
    output_dir="./results/llm-new-categories",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    report_to="none"
)


# Preprocess your dataset to have the expected input format
def preprocess_train_dataset(example):
    return {"text": example["instruction"]}

new_categories_dataset = new_categories_dataset.map(preprocess_train_dataset)

# Initialize SFT trainer
new_trainer = SFTTrainer(
    model=model_for_new_categories,
    train_dataset=new_categories_dataset,
    args=new_training_args,
    peft_config=lora_config
)


def format_instruction(example):
    """Format instruction for training."""
    return example["instruction"]


# Custom tokenize function to handle "instruction" key
def tokenize(example, processing_class):
    processed = processing_class(text=example["instruction"])  # Pass "instruction"
    if (
        processing_class.eos_token_id is not None
        and processing_class.padding_side == "right"
    ):
        processed["labels"] = [
            -100 if i == processing_class.eos_token_id else i
            for i in processed["input_ids"]
        ]
    return processed


# Train the model on new categories
print("Starting training on new categories...")
new_trainer.train()

# Save the fine-tuned model for new categories
model_for_new_categories.save_pretrained("./results/llm-new-categories-final")
tokenizer.save_pretrained("./results/llm-new-categories-final")
print("Model training on new categories completed and saved")

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/15 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training on new categories...


Step,Training Loss


Model training on new categories completed and saved


## 9. Test the Adapted Model on Both Known and New Categories

In [14]:
# Load the model with the new LoRA adapter
new_peft_model_path = "./results/llm-new-categories-final"
new_config = PeftConfig.from_pretrained(new_peft_model_path)


# Configure quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True # Enable CPU offloading
)


# Load model with quantization and CPU offloading
adapted_model = AutoModelForCausalLM.from_pretrained(
    new_config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto"
    )
adapted_model = PeftModel.from_pretrained(adapted_model, new_peft_model_path, device_map="auto")

# Create a text generation pipeline with the adapted model
adapted_pipe = pipeline(
    "text-generation",
    model=adapted_model,
    tokenizer=tokenizer,
    max_new_tokens=20,
    temperature=0.1
)

def format_instruction(product):
    """Format product text into instruction for inference."""
    return f"### Instruction:\nCategorize the following product into one of the standard e-commerce categories.\n\n### Product:\n{product}\n\n### Response:\n"

# Function to predict category with the adapted model
def predict_with_adapted_model(product):
    prompt = format_instruction(product)
    response = pipe(prompt)[0]['generated_text']
    # Extract the category from the response
    category = response.split("### Response:\n")[-1].strip()
    return category

# Test with both known and new categories
combined_test_products = [
    # Known categories
    "Wireless gaming headphones with RGB lighting",
    "Wooden dining table with extendable leaf",
    "Classic leather wallet with coin pocket",
    "Cotton polo shirt with embroidered logo",

    # New categories
    "Smart fitness tracker with heart rate monitor",
    "Electric scooter with foldable design",
    "Organic vitamin supplements in glass bottle",
    "Handcrafted ceramic plant pot with drainage",

    # Additional test cases
    "Bluetooth smartwatch with fitness tracking",
    "Electric unicycle with self-balancing technology",
    "Meditation app subscription with guided sessions",
    "Automatic plant watering system with soil sensors"
]

print("Testing the adapted model on both known and new categories:\n")
for product in combined_test_products:
    category = predict_with_adapted_model(product)
    print(f"Product: '{product}'")
    print(f"Classification: {category}")
    print("-" * 70)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

Testing the adapted model on both known and new categories:

Product: 'Wireless gaming headphones with RGB lighting'
Classification: Electronics

### Instruction:
Categorize the following product into one
----------------------------------------------------------------------
Product: 'Wooden dining table with extendable leaf'
Classification: Furniture

### Instruction:
Categorize the following product into one of
----------------------------------------------------------------------
Product: 'Classic leather wallet with coin pocket'
Classification: ### Instruction:

Categorize the following product into one of the standard e
----------------------------------------------------------------------
Product: 'Cotton polo shirt with embroidered logo'
Classification: ### Instruction:

Categorize the following product into one of the standard e
----------------------------------------------------------------------
Product: 'Smart fitness tracker with heart rate monitor'
Classification: ### Ins

## 10. Implement Continuous Test-time Adaptation

In [15]:
# Function to simulate continuous test-time adaptation
def continuous_tao(product, known_categories, entropy_threshold=5.0, confidence_threshold=0.7):
    prompt = format_instruction(product)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU explicitly


    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        # Calculate entropy on the last token's logits
        last_token_logits = logits[:, -1, :]
        entropy = calculate_entropy(last_token_logits)

        # Generate response
        response = adapted_pipe(prompt)[0]['generated_text']
        category = response.split("### Response:\n")[-1].strip()

        # Check if the predicted category is in known categories
        if category in known_categories:
            confidence = 1.0 - (entropy / 10.0)  # Normalize entropy to a confidence score
            if confidence < confidence_threshold:
                return "Uncertain - Needs Human Review", entropy, confidence
            else:
                return category, entropy, confidence
        else:
            # This is a new category prediction
            if entropy > entropy_threshold:
                return "New Category - High Uncertainty", entropy, 0.0
            else:
                return category, entropy, 0.8  # Reasonable confidence in the new category

# List of known categories
known_categories = ['Electronics', 'Clothing', 'Furniture', 'Accessories', 'Kitchen',
                    'Wearable Tech', 'Transportation', 'Health & Wellness', 'Home & Garden']

# Test products for continuous adaptation
continuous_test_products = [
    # Known categories with clear classification
    "High-end gaming laptop with RTX graphics",
    "Merino wool sweater with turtleneck design",

    # Known categories but unusual descriptions
    "Foldable smartphone with flexible OLED screen",
    "Modular sofa that transforms into a bed",

    # New categories from our adaptation
    "GPS running watch with heart rate monitoring",
    "Collapsible electric scooter with removable battery",

    # Potentially completely new categories
    "NFT digital art collection with blockchain certificate",
    "Virtual reality meditation experience subscription",
    "AI-powered personal shopping assistant service",
    "Biodegradable phone case made from plant materials"
]

print("Testing continuous test-time adaptation:\n")
for product in continuous_test_products:
    category, entropy, confidence = continuous_tao(product, known_categories)
    print(f"Product: '{product}'")
    print(f"Classification: {category}")
    print(f"Uncertainty (Entropy): {entropy:.4f}, Confidence: {confidence:.4f}")
    print("-" * 70)

Testing continuous test-time adaptation:

Product: 'High-end gaming laptop with RTX graphics'
Classification: Computers

### Instruction:
Categorize the following product into one of the
Uncertainty (Entropy): nan, Confidence: 0.8000
----------------------------------------------------------------------
Product: 'Merino wool sweater with turtleneck design'
Classification: Clothing

### Instruction:
Categorize the following product into one of the
Uncertainty (Entropy): nan, Confidence: 0.8000
----------------------------------------------------------------------
Product: 'Foldable smartphone with flexible OLED screen'
Classification: Electronics

### Instruction:
Categorize the following product into one
Uncertainty (Entropy): nan, Confidence: 0.8000
----------------------------------------------------------------------
Product: 'Modular sofa that transforms into a bed'
Classification: Furniture

### Instruction:
Categorize the following product into one of
Uncertainty (Entropy): nan, 

## 11. Conclusion and Next Steps

In this notebook, we've implemented a Test-time Adaptation for Out-of-distribution detection (TAO) system using:

1. A pre-trained LLM (Llama-2) fine-tuned on known product categories
2. LoRA for efficient adaptation to new categories
3. Entropy-based uncertainty estimation for detecting out-of-distribution samples
4. Continuous test-time adaptation for handling evolving product categories

Next steps for improving this system could include:

- Implementing active learning to selectively query human experts for uncertain predictions
- Developing a more sophisticated uncertainty estimation method beyond entropy
- Creating a feedback loop for continuous model improvement
- Implementing a multi-adapter approach to handle different domains separately
- Exploring parameter-efficient fine-tuning methods beyond LoRA (e.g., QLoRA, IA³)